In [20]:
import random

import exdir
import numpy as np

In [21]:
file = exdir.File("mytestfile.exdir")

In [36]:
main_dataset = file['unprocessed']
main_dataset.create_group('train')
main_dataset.create_group('valid')
main_dataset.create_group('test')

<Exdir Group 'mytestfile.exdir/unprocessed/test' (mode a)>

In [40]:
splits = list(dataset.keys())
for split in splits:
    split_dataset = main_dataset[split]
    for i in range(9):
        exam_id = f'LD{random.randint(0, 10)}{random.randint(0, 10)}'
        if exam_id in split_dataset:
            while exam_id in split_dataset:
                exam_id = f'LD{random.randint(0, 10)}{random.randint(0, 10)}'
        else:
            dset = split_dataset.create_dataset(exam_id, data=np.random.rand(7,4,5))
            dset.attrs = {
                'exam_id': exam_id,
                'label': random.randint(0, 2)
            }

In [41]:
main_dataset

<Exdir Group 'mytestfile.exdir/unprocessed' (mode a)>